In [1]:
# Import libraries

import tensorflow as tf
import random
import graphical, game
import numpy as np
import os

from collections import deque, namedtuple

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
N_Rows = 10
N_Cols = 8
N_Dir = 2

def match_in_row_from_index(board_num, i, j, flag_print):
    count_equal_forward = 1
    count_equal_backward = 0
    c1 = board_num[i, j]
    if c1 < 0:
        return False
    
    if j + 1 < N_Cols and c1 == board_num[i, j + 1]:
        count_equal_forward += 1
        if j + 2 < N_Cols and c1 == board_num[i, j + 2]:
            count_equal_forward += 1
    
    if j - 1 >= 0 and c1 == board_num[i, j - 1]:
        count_equal_backward += 1
        if j - 2 >= 0 and c1 == board_num[i, j - 2]:
            count_equal_backward += 1
    if flag_print:
        print(count_equal_backward + count_equal_forward)
    
    if count_equal_backward + count_equal_forward >= 3:
        return True
    return False

def match_in_col_from_index(board_num, i, j, flag_print):
    count_equal_forward = 1
    count_equal_backward = 0
    r1 = board_num[i, j]
    if r1 < 0:
        return False
    
    if i + 1 < N_Rows and r1 == board_num[i + 1, j]:
        count_equal_forward += 1
        if i + 2 < N_Rows and r1 == board_num[i + 2, j]:
            count_equal_forward += 1
    
    if i - 1 >= 0 and r1 == board_num[i - 1, j]:
        count_equal_backward += 1
        if i - 2 >= 0 and r1 == board_num[i - 2, j]:
            count_equal_backward += 1
    
    if flag_print:
        print(count_equal_backward + count_equal_forward)
    
    if count_equal_backward + count_equal_forward >= 3:
        return True
    return False

def swap_col(board_num, row, col):
    tmp = board_num[row, col + 1]
    board_num[row, col + 1] = board_num[row, col]
    board_num[row, col] = tmp
    return board_num

def swap_row(board_num, row, col):
    tmp = board_num[row + 1, col]
    board_num[row + 1, col] = board_num[row, col]
    board_num[row, col] = tmp
    return board_num

def get_board_num(board):
    #print(board)
    board_num = np.zeros((N_Rows, N_Cols))
    col = 0
    row = 0
    for s in range(0, len(board)):
        if board[s] != '\n':
            board_num[row, col] = ord(board[s]) - ord('a')
            board_num[row, col] = -1 if board_num[row, col] < 0 else board_num[row, col]
            
            col += 1
        else:
            col = 0
            row += 1
    #print(board_num)
    return board_num

def get_game_state(board, moves_left):
    board_num = get_board_num(board)
    
    flag_print = False
    col_feature = np.zeros((N_Rows, N_Cols))
    for row in range(0, N_Rows):
        for col in range(0, N_Cols - 1):
            #flag_print = False
            #if row == 3 and col == 3:
            #    flag_print = True
            
            board_num = swap_col(board_num, row, col)
            if flag_print:
                print(board_num)
            if (match_in_row_from_index(board_num, row, col, flag_print) or
                match_in_col_from_index(board_num, row, col, flag_print) or
                match_in_row_from_index(board_num, row, col + 1, flag_print) or
                match_in_col_from_index(board_num, row, col + 1, flag_print)):
                col_feature[row, col] = 1
            board_num = swap_col(board_num, row, col)
            if flag_print:
                print(board_num)
    #print(col_feature)
    
    row_feature = np.zeros((N_Rows, N_Cols))
    for row in range(0, N_Rows - 1):
        for col in range(0, N_Cols):
            #flag_print = False
            #if row == 3 and col == 3:
            #    flag_print = True
            
            board_num = swap_row(board_num, row, col)
            if flag_print:
                print(board_num)
            if (match_in_row_from_index(board_num, row, col, flag_print) or
                match_in_col_from_index(board_num, row, col, flag_print) or
                match_in_row_from_index(board_num, row + 1, col, flag_print) or
                match_in_col_from_index(board_num, row + 1, col, flag_print)):
                row_feature[row, col] = 1
            board_num = swap_row(board_num, row, col)
            if flag_print:
                print(board_num)
    #print(row_feature)
    
    ###### put features in the state
    state = np.zeros(2 * (N_Rows * N_Cols))
    
    c_state_index = 0
    for row in range(0, N_Rows):
        for col in range(0, N_Cols):
            v = 0
            if col_feature[row, col] and row_feature[row, col]:
                v = 1
            elif col_feature[row, col]:
                v = 2
            else:
                v = 3
            state[c_state_index] = 2.0 * (v / 3.0) - 1.0
            c_state_index += 1
            state[c_state_index] = 2.0 * ((board_num[row, col] + 1) / 5.0) - 1.0
            c_state_index += 1

    #state[c_state_index] = moves_left / 25.0
    
    return state

def get_action_from(move):
    action = np.array(move)
    
    if move[2]:
        action[2] = 1
    else:
        action[2] = 0
    
    out_action = (action[2]) * (N_Rows * N_Cols) + (action[0] * N_Rows + action[1])
    
    return out_action

def get_move_from(action):
    row_col = action % (N_Rows * N_Cols)
    
    dir = int(action / (N_Rows * N_Cols))
    
    return (int(row_col / N_Rows), row_col % N_Rows, dir >= 1)

def get_valid_actions():
    All_Actions = np.arange(0,N_Rows*N_Cols*N_Dir)
    #print(All_Actions)
    
    remove_ids = []
    for i in range(0, N_Rows):
        action_id = get_action_from((N_Cols - 1, i, False))
        remove_ids.append(action_id)
    
    for i in range(0, N_Cols):
        action_id = get_action_from((i, N_Rows - 1, True))
        remove_ids.append(action_id)
    print("removed actions:", remove_ids)
    return np.delete(All_Actions, remove_ids, axis=0), remove_ids

All_Actions, remove_ids = get_valid_actions()
#print(All_Actions)

board_sample_str = "da#bb#ac\n#bbccbbd\n#cd#a#d#\n#c#d#ddc\n##ba##bc\nacadc#c#\n#d##cc##\nc#cbdacd\ndca#d#b#\ndd#dccdb"
state = get_state(board_sample_str, 0)

N_All_Actions = len(All_Actions)
N_State = len(state)

###################################################### test ############################################

# test action conversion
num_error_in_conversion = 0
for i in range(0,160):
    a = get_move_from(i)
    ii = get_action_from(a)
    if i != ii:
        num_error_in_conversion += 1
print("number of errors happens in action conversion: ", num_error_in_conversion)

print("removed for example:", get_move_from(159), get_move_from(73))

print("Num of state: ", N_State, ", Num of actions: ", N_All_Actions)

removed actions: [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 89, 99, 109, 119, 129, 139, 149, 159]
number of errors happens in action conversion:  0
removed for example: (7, 9, True) (7, 3, False)
Num of state:  160 , Num of actions:  142


In [ ]:
class Model():
    def __init__(self, 
                 #summaries_dir, 
                 #policy_type, 
                 action_size, 
                 state_size, 
                 max_steps, 
                 #use_autoencoder_as_feature_selection,
                 global_step):
        #if summaries_dir:
        #    if not os.path.exists(summaries_dir):
        #        os.makedirs(summaries_dir)
        #    self.summary_writer = tf.summary.FileWriter(summaries_dir)
        
        self.global_step = global_step
        self.max_steps = max_steps
        self.action_size = action_size
        self.state_size = state_size
        
        self.state = tf.placeholder(shape=[None, N_State], dtype=tf.float32, name="state")
        #self.next_state = tf.placeholder(shape=[None, N_State], dtype=tf.float32, name="next_state")
        self.action = tf.placeholder(shape=[None], dtype=tf.float32, name="action")
        
        #self.activation = tf.nn.relu
        #self.policy_type = policy_type
        
        # Policy and value network        
        #self.use_autoencoder_as_feature_selection = use_autoencoder_as_feature_selection
        self.model_summaries = {'learning_rate':deque(maxlen=500),
                                'beta':deque(maxlen=500),
                                'policy_loss':deque(maxlen=500),
                                'entropy':deque(maxlen=500),
                                'value_loss':deque(maxlen=500)}
        #if self.use_autoencoder_as_feature_selection:
        #    encoded_state = self.u_net(input_image=self.state, reuse=False, scope='')
        #    #encoded_next_state = self.u_net(input_image=self.next_state, reuse=True, scope='')
        #    self.PolicyEstimator(tf.stop_gradient(encoded_state))
        #    self.ValueEstimator(tf.stop_gradient(encoded_state))
        #else:
        encoded_state = self.state_processor(input_X=self.state)
        self.PolicyEstimator(encoded_state)
        self.ValueEstimator(encoded_state)

        self.define_loss()
        self.set_graph=False
    
    def state_processor(self, input_X):
        hidden_layer_size = 64
        with tf.variable_scope("encoded_state"):
            fully1 = tf.layers.dense(input_X, hidden_layer_size, activation=tf.nn.relu)      
            fully2 = tf.layers.dense(fully1, hidden_layer_size, activation=tf.nn.relu) 
            fully3 = tf.layers.dense(fully2, hidden_layer_size, activation=tf.nn.relu)
        return fully3
    
    def PolicyEstimator(self, encoded_state):
        use_continuous = True
        with tf.variable_scope("policy_estimator"):
            
            self.advantage = tf.placeholder(
                shape=[None],
                dtype=tf.float32,
                name="advantage"
            )
            
            if use_continuous:
                mu = tf.layers.dense(
                    encoded_state,
                    self.action_size,
                    activation=None,
                    kernel_initializer=LearningModel.scaled_init(0.01),
                    reuse=tf.AUTO_REUSE,
                )

                self.log_sigma_sq = tf.get_variable(
                    "log_sigma_squared",
                    [self.action_size],
                    dtype=tf.float32,
                    initializer=tf.zeros_initializer(),
                )

                sigma_sq = tf.exp(self.log_sigma_sq)

                self.epsilon = tf.placeholder(
                    shape=[None, self.action_size], dtype=tf.float32, name="epsilon"
                )
                # Clip and scale output to ensure actions are always within [-1, 1] range.
                self.output_pre = mu + tf.sqrt(sigma_sq) * self.epsilon
                output_post = tf.clip_by_value(self.output_pre, -3, 3) / 3
                self.output = tf.identity(output_post, name="action")
                self.selected_actions = tf.stop_gradient(output_post)
                # Compute probability of model output.
                all_probs = (
                    -0.5 * tf.square(tf.stop_gradient(self.output_pre) - mu) / sigma_sq
                    - 0.5 * tf.log(2.0 * np.pi)
                    - 0.5 * self.log_sigma_sq
                )

                self.all_log_probs = tf.identity(all_probs, name="action_probs")

                self.entropy = 0.5 * tf.reduce_mean(
                    tf.log(2 * np.pi * np.e) + self.log_sigma_sq
                )

                self.all_old_log_probs = tf.placeholder(
                    shape=[None, self.act_size[0]], dtype=tf.float32, name="old_probabilities"
                )

                # We keep these tensors the same name, but use new nodes to keep code parallelism with discrete control.
                self.log_probs = tf.reduce_sum(
                    (tf.identity(self.all_log_probs)), axis=1, keepdims=True
                )
                self.old_log_probs = tf.reduce_sum(
                    (tf.identity(self.all_old_log_probs)), axis=1, keepdims=True
                )
            else:
                self.old_action_probs = tf.placeholder(shape=[None, self.action_size], 
                                                       dtype=tf.float32, 
                                                       name="old_action_probabilities")

                self.action_probs = tf.layers.dense(encoded_state, 
                                                    self.action_size,
                                                    activation=tf.nn.softmax,
                                                    kernel_initializer=tf.contrib.layers.variance_scaling_initializer(0.01))

                self.picked_action_prob = tf.reduce_sum(self.action_probs *\
                                                        tf.one_hot(self.action, self.action_size),
                                                        axis=1)
                self.picked_old_action_prob = tf.reduce_sum(self.old_action_probs *\
                                                            tf.one_hot(self.action, self.action_size),
                                                            axis=1)
                self.entropy = -tf.reduce_sum(self.action_probs *\
                                              tf.log(self.action_probs + 1e-10),
                                              axis=1)
                self.mean_entropy = tf.reduce_mean(self.entropy)

                self.decay_epsilon = tf.train.polynomial_decay(0.2, self.global_step, self.max_steps, 0.1, power=1.0)
                #Clipped Surrogate Objective
                ratio = self.picked_action_prob / (self.picked_old_action_prob  + 1e-10)
                a = ratio * self.advantage
                b = tf.clip_by_value(ratio, 1.0 - self.decay_epsilon, 1.0 + self.decay_epsilon) * self.advantage
                self.policy_loss = -tf.reduce_mean(tf.minimum(a, b))
                self.abs_policy_loss = abs(self.policy_loss)
            
    def policy_predict(self, sess, state):
        if not self.set_graph:
            self.summary_writer.add_graph(sess.graph)
            self.set_graph = True
        return sess.run(self.action_probs, {self.state: state})
    
    def ValueEstimator(self, encoded_state):
        with tf.variable_scope("value_estimator"):
            self.target = tf.placeholder(shape=[None], dtype=tf.float32, name="target")
            self.old_value = tf.placeholder(shape=[None], dtype=tf.float32, name="old_values")
            self.value_estimate = tf.layers.dense(encoded_state, 1, activation=None)
            self.value_estimate = tf.squeeze(self.value_estimate, axis=1)
            
            a = tf.squared_difference(self.value_estimate, self.target)
            clipped_value_estimate = self.old_value + tf.clip_by_value(self.value_estimate - self.old_value , 
                                                                       -self.decay_epsilon, self.decay_epsilon)
            b = tf.squared_difference(clipped_value_estimate, self.target)
            self.value_loss = tf.reduce_mean(tf.minimum(a, b))
            
    def value_predict(self, sess, state):
        return sess.run(self.value_estimate, {self.state: state})
    
    def define_loss(self, learning_rate=1e-4):
        self.decay_learning_rate = tf.train.polynomial_decay(learning_rate, self.global_step, self.max_steps, 1e-10, power=1.0)
        self.decay_beta = tf.train.polynomial_decay(5e-3, self.global_step, self.max_steps, 1e-5, power=1.0)
        
        self.loss = self.policy_loss + 0.5 * self.value_loss - self.decay_beta * tf.reduce_mean(self.entropy)
        optimizer = tf.train.AdamOptimizer(learning_rate=self.decay_learning_rate)
        self.train_op = optimizer.minimize(self.loss)
            
    def update_model(self, sess, state, next_state, action, old_action_probs, target, advantage, old_value):
        feed_dict = {self.state: state, self.next_state: next_state, 
                     self.action: action, self.old_action_probs: old_action_probs, 
                     self.target: target, self.advantage: advantage, self.old_value: old_value}
        
        lr, beta, abs_policy_loss, entropy, value_loss, _, _, loss = \
            sess.run([self.decay_learning_rate, self.decay_beta, 
                      self.abs_policy_loss, self.mean_entropy,
                      self.value_loss, tf.train.get_global_step(), 
                      self.train_op, self.loss], feed_dict)
        self.model_summaries['learning_rate'].append(lr)
        self.model_summaries['beta'].append(beta)
        self.model_summaries['policy_loss'].append(abs_policy_loss)
        self.model_summaries['entropy'].append(entropy)
        self.model_summaries['value_loss'].append(value_loss)

In [ ]:
class Agent(object):
    def __init__(self, 
                 max_steps=10e+6, 
                 total_episodes=100000, 
                 discount_factor=0.99):
        
        tf.reset_default_graph()
        
        global_step_tensor = tf.Variable(0, name="global_step", trainable=False)
        sess = tf.Session()
        sess.run(global_step_tensor.initializer)
    
        state_size = N_State
        action_size = 3
        self.sess = sess
        #self.model = Model(summaries_dir='./experiments_' + policy_type + '_' + env_name + '/summaries', 
        #                   policy_type=policy_type, action_size=action_size, state_size=state_size,
        #                   max_steps=max_steps, use_autoencoder_as_feature_selection=use_autoencoder_as_feature_selection, 
        #                   global_step=global_step_tensor)
        
        self.model = Model(action_size=action_size,
                           state_size=state_size,
                           max_steps=max_steps, 
                           global_step=global_step_tensor)
        
        #first build the model, then initialize variables
        self.sess.run(tf.global_variables_initializer())
        self.total_episodes = total_episodes
        self.max_replay_memory_size = 10240
        self.batch_size = 1024
        self.epochs = 3
        self.discount_factor = discount_factor
        self.lambda_ = 0.95
        
        self.checkpoint_dir = os.path.join('./experiments', 'checkpoints')
        self.checkpoint_path = os.path.join(self.checkpoint_dir, 'model')
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)
        self.saver = tf.train.Saver()
        self.load_model(self.checkpoint_dir)        
    
    def save_model():
        pass
    
    def load_model(self):
        self.latest_checkpoint = tf.train.latest_checkpoint(self.checkpoint_dir)
        if self.latest_checkpoint:
            print("Loading checkpoint")
            self.saver.restore(self.sess, self.latest_checkpoint)
    
    def get_action(self, state):
        action_prob = self.model.policy_predict(self.sess, np.expand_dims(state,axis=0))[0]
        action = np.random.choice(np.arange(len(action_prob)), p=action_prob)
        
        return action, action_prob
    
    def predict_action(self, board, score, moves_left):
        
        state = get_game_state(board, moves_left)
        
        action, action_prob = self.get_action(state)
        
        pass
    
    def collect_observation(self,board, move, score_delta, next_board, moves_left):
        pass
    
    def update(self):
        pass
    
    def init_episode(self):
        pass

In [ ]:
global my_dqn

def ai_callback(board, score, moves_left):
    global my_dqn
    
    predicted_move = my_dqn.predict_action(board, score, moves_left)
    #print(predicted_move)
    #dir = random.randint(0, 1) == 0
    #return (random.randint(0, 7 if dir else 6), random.randint(0, 8 if dir else 9), dir)
    return predicted_move

def transition_callback(board, move, score_delta, next_board, moves_left):
    global my_dqn
    
    my_dqn.collect_observation(board, move, score_delta, next_board, moves_left)
    my_dqn.update()
    
    pass # This can be used to monitor outcomes of moves

def end_of_game_callback(boards, scores, moves, final_score):
    global my_dqn
    
    my_dqn.init_episode()
    
    return True # True = play another, False = Done


if __name__ == '__main__':
    global my_dqn
    
    tf.reset_default_graph()

    # Where we save our checkpoints and graphs
    experiment_dir = os.path.abspath("./experiments/{}".format("ubisoft-game"))

    # Create a glboal step variable
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Create estimators
    q_estimator = QNetwork(init_learning_rate=1e-3, scope="q_estimator", summaries_dir=experiment_dir)
    
    target_estimator = QNetwork(scope="target_q")
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    my_dqn = deep_q_learning(sess,
                             q_estimator=q_estimator,
                             target_estimator=target_estimator,
                             experiment_dir=experiment_dir,
                             num_episodes=50000,
                             replay_memory_size=1000000,
                             epsilon_start=1.0,
                             epsilon_end=0.1,
                             epsilon_decay_steps=500000,
                             discount_factor=0.99,
                             batch_size=64)

    speedup = 1000.0
    g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
    g.run()


Instructions for updating:
Please switch to tf.train.get_global_step
Loading model checkpoint C:\Kourosh\Project\Ubisoft\RedLynx_ML_Assignment\experiments\ubisoft-game\checkpoints\model...

INFO:tensorflow:Restoring parameters from C:\Kourosh\Project\Ubisoft\RedLynx_ML_Assignment\experiments\ubisoft-game\checkpoints\model
Seed: 275554838359217849107027030859888925544
Step 16 (806359) @ Episode 32254/50000, loss: 0.88576990365982064